<a href="https://colab.research.google.com/github/thowley1207/capstone_project/blob/main/03_generate_event_study_subset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade wrds
!wget https://raw.githubusercontent.com/thowley1207/capstone_project/main/colab_initialization/initializer.py

import json
import pandas as pd
import pathlib
import numpy as np
import requests
import zipfile

import initializer
initializer.initialize_colab()
db = initializer.initialize_wrds_connection()

In [ ]:
'''
SET DATA SUBDIRECTORIES AND FORM TYPE PREFIX
WHEN APPLICABLE, THIS FORM TYPE PREFIX WILL BE USED MOVING FORWARD
'''
data_subdir = 'data/edgar_wrds_linking/'
file_prefix = '8k_'

'''
ADDITIONAL FILE NAMES CARRIED DOWN FROM PRIOR WORK
'''
filtered_index_data_w_permno_file_name = (
    'filtered_index_data_w_permno.pkl')

'''
NEW FILE NAMES FOR USE BELOW
'''
event_subset_file_name = 'event_subset.pkl'

In [ ]:
'''
READ IN COMBINED FILTERED INDEX DATA W/PERMNO
'''

filtered_index_data_w_permno = pd.read_pickle((
    data_subdir +
    file_prefix +
    filtered_index_data_w_permno_file_name
    ))

### **QUERY HELPER FUNCTION**

In [ ]:
def execute_wrds_query(query_content,
                       wrds_session = db,
                       output_directory = 'data/',
                       output_file = None):

    query_result = db.raw_sql(query_content)

    if output_file is not None:
        output_path = f"""{output_directory}{output_file}"""
        print(f"""Writing query result to: {output_path}""")

        if output_file.endswith('.csv'):
            query_result.to_csv(output_path)
        elif output_file.endswith('.pkl'):
            query_result.to_pickle(output_path)
        else:
            raise Exception("Invalid File Format Provided For Output")
        print(f"""Query result successfully written.""")
    else:
        print(f"""Warning: output is not saved to Google Drive.""")

    return query_result

## **DATA WRANGLING**

 **Step 1:**

* **Create the event study date calendars for all possible event dates**
    * To do so, we utilize CRSP dsf to obtain all distinct dates in the date range being considered; this provides us a calendar of all trading dates in the period (note: not calendar dates)
    * Estimation window set to begin 252 trading days prior to each event date (as there are approximately 252 trading days per year)
    * Estimation window set to end 60 days prior to each event date (as there are approximately 60 trading days in a quarter, thus providing an approx. 1 quarter gap between estimation window and event)
    * Set event window start and end dates for two event window sizes: one 3 day window centered around the event, and one 10 day window centered around the event

In [ ]:
q_event_dates = f"""
with
distinct_dates as
(
    select distinct
        date as event_date
    from crsp.dsf
    where date between '2004-01-01' and '2020-01-15'
),
event_study_dates as
(
    select
        event_date,
        lag(event_date, 252) over (order by event_date asc)
            as est_per_start,
        lag(event_date, 60) over (order by event_date asc)
            as est_per_end,
        lag(event_date, 5) over (order by event_date asc)
            as event_wind_start,
        lead(event_date, 5) over (order by event_date asc)
            as event_wind_end
    from distinct_dates
)
select *
from event_study_dates
where event_date between '2005-01-01' and '2019-12-31'
order by event_date;"""

event_dates = execute_wrds_query(q_event_dates)

for col in event_dates.columns:
    event_dates[col] = pd.to_datetime(event_dates[col])

 **Step 2:**

* **Create a dataframe merging the event study calendar created above with the relevant data from the file containing Edgar index data matched to PERMNOs**
    * Before the merge, drop any columns not needed for the event study component - keep PERMNO, event date, and event period (YYYYMM)
    * In Step 3, we'll only retain data related to events that do not have missing return data in CRSP for the corresponding PERMNO during the estimation and event windows
    * Additionally, we will merge in this step on event date, which allows only for event dates that occur on trading days

In [ ]:
filtered_index_data_w_permno = filtered_index_data_w_permno.filter(
    ['event_id','period', 'permno', 'event_date'])

index_permno_event_dates = filtered_index_data_w_permno \
    .merge(event_dates, on='event_date')

index_permno_event_dates_dict = {x: index_permno_event_dates[
    index_permno_event_dates['period']==x
    ] for x in index_permno_event_dates.period.unique()}

events_no_missing_data_lst = []

In [ ]:
for k,v in index_permno_event_dates_dict.items():
    # Convert datetime values to string to facilitate using as json recordset
    json_index_permno_event_dates = v.copy()

    for col in json_index_permno_event_dates.columns:
        if json_index_permno_event_dates[col].dtype == 'datetime64[ns]':
            json_index_permno_event_dates[
                col] = json_index_permno_event_dates[col].astype(str)

    # Convert pandas dataframe to JSON for use in query
    json_estimation_window = json_index_permno_event_dates[
        ['event_id',
         'period',
         'permno',
         'est_per_start',
         'est_per_end',
         'event_wind_start',
         'event_wind_end']].to_json(orient="records")

    q_events_no_missing_data = f"""
with
windows_data as
(select
    x.event_id,x.permno,x.est_per_start,
    x.est_per_end,x.event_wind_start,x.event_wind_end
 from json_to_recordset('{json_estimation_window}') as x(
            event_id int,permno float,est_per_start date,
            est_per_end date,event_wind_start date,event_wind_end date)),

crsp_daily as
(select
    permno,date as date_ret,ret
 from crsp_a_stock.dsf
 where ret is not null and permno in (select distinct permno from windows_data)
    and date between (select min(est_per_start) from windows_data)
        and (select max(event_wind_end) from windows_data)),

estimation_window_data as
(select
    events.event_id,events.permno,crsp.date_ret,crsp.ret
 from windows_data events left join crsp_daily crsp
    on events.permno = crsp.permno
    and crsp.date_ret between events.est_per_start and events.est_per_end),

event_period_data as
(select
    events.event_id,crsp.ret
 from windows_data events left join crsp_daily crsp
    on events.permno = crsp.permno
    and crsp.date_ret between events.event_wind_start and events.event_wind_end),

estimation_window_counts as
(select
    event_id, count(ret) as num_returns_est_window
 from estimation_window_data group by 1),

event_window_counts as
(select
    event_id, count(ret) as num_returns_event_window
 from event_period_data group by 1)

select
    coalesce(est.event_id, event.event_id) as event_id,
    est.num_returns_est_window, event.num_returns_event_window
from estimation_window_counts est join event_window_counts event
    on est.event_id = event.event_id
where est.num_returns_est_window = 193 and event.num_returns_event_window = 11;
"""

    events_no_missing_data_per = execute_wrds_query(q_events_no_missing_data)
    events_no_missing_data_lst.append(events_no_missing_data_per)

events_no_missing_data = pd.concat(events_no_missing_data_lst
                                   ).sort_values(by=['event_id'])

event_subset = index_permno_event_dates.merge(events_no_missing_data,
                                              on='event_id'
                                              ).drop(columns = [
                                                  'num_returns_est_window',
                                                  'num_returns_event_window'])

In [ ]:
event_subset.to_pickle((
    data_subdir +
    file_prefix +
    event_subset_file_name
    ))